In [5]:
from pyfirmata import Arduino
import serial
import time

In [12]:
class SimpleFeeder(Arduino):
    pin_diode = 12
    pin_feeder = 8
    
    def __init__(self, *args, **kwargs):
        super(SimpleFeeder, self).__init__(*args, **kwargs)
        self.digital[SimpleFeeder.pin_feeder].write(0)
        self.digital[SimpleFeeder.pin_diode].write(0)

    def blink(self):
        self.digital[SimpleFeeder.pin_diode].write(1)
        time.sleep(1)
        self.digital[SimpleFeeder.pin_diode].write(0)        

    def on(self):
        self.digital[SimpleFeeder.pin_diode].write(1)
        
    def off(self):
        self.digital[SimpleFeeder.pin_diode].write(0)
        
    def feed(self):
        self.digital[SimpleFeeder.pin_feeder].write(1)
        time.sleep(0.01)
        print(0.01)
        self.digital[SimpleFeeder.pin_feeder].write(0)

In [9]:
class MCSArduino(Arduino):
    pin_diode = 13
    pin_TTL_1 = 6
    pin_TTL_2 = 5
    pin_LED_lights = 7
    pin_feeder = 8
    
    def __init__(self, *args, **kwargs):
        self.last_cmd = False  # False - Arduino LOW, True - Arduino HIGH
        self.is_light_off = False
        super(MCSArduino, self).__init__(*args, **kwargs)
        
    def start_or_stop(self):  # acquisition
        self.last_cmd = not self.last_cmd
        self.digital[MCSArduino.pin_diode].write(self.last_cmd)
        self.digital[MCSArduino.pin_TTL_1].write(self.last_cmd)
        self.digital[MCSArduino.pin_TTL_2].write(self.last_cmd)

    def switch_light(self):
        self.is_light_off = not self.is_light_off
        self.digital[MCSArduino.pin_LED_lights].write(self.is_light_off)
        
    def feed(self):
        self.digital[MCSArduino.pin_feeder].write(True)
        time.sleep(0.02)
        self.digital[MCSArduino.pin_feeder].write(False)
        
        
class FakeArduino():
    def __init__(self):
        self.is_light_off = False
        
    def start_or_stop(self):
        print("Fake Arduino - sending a TTL pulse")
        
    def exit(self):
        print("Fake Arduino - exiting...")
        
    def switch_light(self):
        self.is_light_off = not self.is_light_off
        print("Fake Arduino - switching light on/off...")
        
    def feed(self):
        print("Fake Arduino - feeding...")

In [3]:
class FakeDevice:
    def write(self):
        pass
    def close(self):
        pass

class Feeder:
    def __init__(self, port, baudrate=9600):
        if port == 'fake':
            self.device = FakeDevice()
        else:
            self.device = serial.Serial(port, baudrate=baudrate)
        
    def feed(self):
        self.device.write('SP200\r'.encode())
        self.device.write('LR6175\r'.encode())
        self.device.write('m\r'.encode())
        
    def exit(self):
        self.device.close()

In [4]:
class ArduinoDevice(object):
    """
    NOTE for LINUX users! Include user in the 'dialout' group and restart to
    access USB devices.
    """

    def __init__(self, port=None, baud=9600):
        self.device = serial.Serial(port, baud, timeout=.1)

    @staticmethod
    def serial_ports():
        """
        Lists available serial ports

        :raises EnvironmentError:
            On unsupported or unknown platforms
        :returns:
            A list of the serial ports available on the system
        """
        return comports()

    def read(self):
        return self.device.readline()

    def write(self, data):
        return self.device.write(data)

    def close(self):
        self.device.close()


### Testing Arduino TTL pulses

In [19]:
board = MCSArduino('COM10')  # Windows - 'COM10', Linux - '/dev/ttyACM0', check /dev/tty*

In [10]:
# check the light on-off
board.switch_light()

In [23]:
# testing sync with acquisition system
board.start_or_stop()

In [7]:
board.feed()

In [25]:
board.digital[8].write(0)

In [27]:
board.digital[13].write(True)

In [22]:
board.digital[8].write(1)
board.digital[13].write(1)

In [16]:
for i in range(10):
    time.sleep(5)
    board.feed()

In [5]:
# Arduino should blink 2 times
for i in range(4):
    board.start_or_stop()
    time.sleep(2)

In [14]:
board.digital[5].value

False

In [17]:
board.exit()

## Testing blinking

In [15]:
board = SimpleFeeder('COM5')

In [24]:
board.on()

In [25]:
board.off()

In [26]:
board.exit()

### Testing feeder

In [7]:
feeder = SimpleFeeder('COM6')

In [11]:
feeder.feed()

In [7]:
feeder.exit()

In [4]:
feeder = Feeder('COM8')

time.sleep(5)  # time to get inside the chamber and see that it works
feeder.feed()

feeder.exit()

### Motors - speaker position calibration

In [3]:
class SpeakerMotor:
    """
    example command to the arduino: +20:600
    
    +/-   direction
    20    number of steps to rotate
    600   speed in steps per second
    """
    
    def __init__(self, port, baud=9600):
        self.port = port
        self.baud = baud
        self.diodes_on = False
        self.device = serial.Serial(port, baud, timeout=.1)

    def microstep_left(self):
        self.device.write('-10:200\n'.encode('utf-8'))  # 90deg is 17120 steps
        #time.sleep(0.05)
        #print(self.device.readline())

    def microstep_right(self):
        self.device.write('+10:200\n'.encode('utf-8'))

    def switch_diodes(self):
        self.device.write('d\n'.encode('utf-8'))
        self.diodes_on = not self.diodes_on
        
    def rotate(self, direction, degrees, duration):
        # direction True CCW, False CW
        # degrees in deg, 45, 90 etc.
        # duration in sec
        speed = 190 * (degrees / duration)
        steps = 190 * degrees
        
        self.turn(1 if direction else -1, int(steps), int(speed))
        time.sleep(0.1)
        
    def turn(self, direction, steps, speed):
        assert type(steps) == type(1)
        assert type(speed) == type(1)
        assert speed > 0
        assert speed < 1200
        assert steps < 20000
        assert abs(direction) == 1
        
        # +CCW, -CW
        command = '%s%s:%s\n' % ('+' if direction > 0 else '-', str(steps), str(speed))
        print('sending ' + command)
        self.device.write(command.encode('utf-8'))
        
    def exit(self):
        self.device.close()

In [2]:
class CableMotor:
    """
    example command to the arduino: +800:600
    
    +/-   direction
    800    number of steps to rotate (1600 is one turn)
    600   speed in steps per second
    """
    
    def __init__(self, port, baud=9600):
        self.port = port
        self.baud = baud
        self.diodes_on = False
        self.device = serial.Serial(port, baud, timeout=.1)

    def turn_CW(self):
        self.device.write('+1600:800\n'.encode('utf-8'))

    def turn_CCW(self):
        self.device.write('-1600:800\n'.encode('utf-8'))

    def exit(self):
        self.device.close()

In [34]:
import keyboard
import serial
import time

# r - rotate right, t - rotate left, q - quit
sm = SpeakerMotor('COM12')
out1 = ''

try:
    while True:
        if keyboard.is_pressed('d'):  # turn diodes on / off
            sm.device.write('d\n'.encode('utf-8'))
        
        if keyboard.is_pressed('r'):  # microstepping right
            sm.microstep_right()
            time.sleep(0.1)
        if keyboard.is_pressed('t'):  # microstepping left
            sm.microstep_left()
            time.sleep(0.1)

        if keyboard.is_pressed('u'):  # slow motion
            sm.turn(-1, 17120, 60)
            time.sleep(0.1)
        if keyboard.is_pressed('y'):  # slow motion
            sm.turn(1, 17120, 60)
            time.sleep(0.1)

        if keyboard.is_pressed('i'):  # fast motion
            sm.turn(-1, 17120, 200)
            time.sleep(0.1)
        if keyboard.is_pressed('o'):  # fast motion
            sm.turn(1, 17120, 200)
            time.sleep(0.1)

        if keyboard.is_pressed('m'):
            print("Still running the thread")
            time.sleep(0.1)
            
        if keyboard.is_pressed('q'):
            print("q pressed, ending loop")
            break
                
finally:
    sm.exit()

q pressed, ending loop


In [40]:
board.exit()